# Model Development - Plant Disease Classification

**Objective**: Build, train, and evaluate a deep learning model for plant disease classification using transfer learning.

**ML Lifecycle Stages**:
1. Problem Definition & Requirements
2. Data Preparation & Loading
3. Model Architecture Design
4. Training Strategy & Configuration
5. Model Training & Monitoring
6. Performance Evaluation
7. Hyperparameter Tuning & Optimization
8. Final Model Selection & Validation

---

This notebook demonstrates the complete end-to-end machine learning workflow, from initial problem formulation through final model evaluation.

## 1. Problem Definition & Requirements

### Business Problem
Agricultural productivity is significantly impacted by plant diseases. Early and accurate detection can:
- Reduce crop losses
- Minimize pesticide usage
- Enable targeted treatment
- Improve yield quality

### Technical Problem
**Task**: Multi-class image classification  
**Input**: RGB images of plant leaves (224×224 pixels)  
**Output**: Disease class prediction (16 classes)  
**Success Metrics**: 
- Accuracy > 85% (target)
- Balanced performance across classes (F1-score > 0.80)
- Fast inference time (< 1 second per image)

### Constraints
- **Hardware**: CPU-only training (no GPU)
- **Dataset**: PlantVillage dataset (~20K images)
- **Deployment**: Must be lightweight enough for mobile/edge devices

## 2. Setup & Imports

In [ ]:
import sys
import os
from pathlib import Path
import yaml
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, models

# Add project source to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## 3. Configuration Management

Load project configuration from `config.yaml` for reproducibility and easy experimentation.

In [ ]:
# Load configuration
config_path = Path.cwd().parent / 'config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Display key configuration
print("📋 Project Configuration:")
print("="*70)
print(f"Model: {config['model']['architecture']}")
print(f"Number of classes: {config['model']['num_classes']}")
print(f"Image size: {config['data']['image_size']}")
print(f"Batch size: {config['training']['batch_size']}")
print(f"Learning rate: {config['training']['learning_rate']}")
print(f"Number of epochs: {config['training']['num_epochs']}")
print(f"Optimizer: {config['training']['optimizer']}")
print(f"Scheduler: {config['training']['scheduler']}")
print("="*70)

## 4. Data Loading & Preparation

### 4.1 Dataset Statistics

In [ ]:
# Check preprocessed data
from preprocessing.data_loader import PlantDiseaseDataset

train_dir = Path.cwd().parent / 'data' / 'processed' / 'train'
val_dir = Path.cwd().parent / 'data' / 'processed' / 'val'
test_dir = Path.cwd().parent / 'data' / 'processed' / 'test'

# Count images in each split
def count_images(data_dir):
    total = 0
    class_counts = {}
    for class_dir in data_dir.iterdir():
        if class_dir.is_dir() and not class_dir.name.startswith('.'):
            count = len(list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.JPG')))
            class_counts[class_dir.name] = count
            total += count
    return total, class_counts

train_total, train_classes = count_images(train_dir)
val_total, val_classes = count_images(val_dir)
test_total, test_classes = count_images(test_dir)

print("📊 Dataset Split Statistics:")
print("="*70)
print(f"Training set:   {train_total:,} images ({train_total/(train_total+val_total+test_total)*100:.1f}%)")
print(f"Validation set: {val_total:,} images ({val_total/(train_total+val_total+test_total)*100:.1f}%)")
print(f"Test set:       {test_total:,} images ({test_total/(train_total+val_total+test_total)*100:.1f}%)")
print(f"Total:          {train_total+val_total+test_total:,} images")
print(f"Number of classes: {len(train_classes)}")
print("="*70)

### 4.2 Data Augmentation Strategy

Data augmentation is crucial for:
1. **Preventing overfitting** - Increases effective training set size
2. **Improving generalization** - Model learns invariant features
3. **Handling variations** - Rotation, lighting, scale variations in real-world deployment

In [ ]:
# Import augmentation module
from preprocessing.augmentation import get_training_transforms, get_validation_transforms

# Get transforms
train_transform = get_training_transforms(
    image_size=config['data']['image_size'],
    advanced=True  # Use Albumentations for advanced augmentation
)

val_transform = get_validation_transforms(
    image_size=config['data']['image_size']
)

print("🔄 Data Augmentation Pipeline:")
print("\nTraining Augmentations:")
print("  ✓ Random rotation (±20°)")
print("  ✓ Horizontal flip (p=0.5)")
print("  ✓ Vertical flip (p=0.3)")
print("  ✓ Brightness/Contrast adjustment")
print("  ✓ Gaussian blur (p=0.2)")
print("  ✓ RGB shift")
print("  ✓ Normalization (ImageNet stats)")
print("\nValidation/Test:")
print("  ✓ Resize to 224×224")
print("  ✓ Normalization only (no augmentation)")

### 4.3 Create DataLoaders

In [ ]:
from preprocessing.data_loader import create_dataloaders

# Create dataloaders
train_loader, val_loader, test_loader, class_names = create_dataloaders(
    train_dir=str(train_dir),
    val_dir=str(val_dir),
    test_dir=str(test_dir),
    batch_size=config['training']['batch_size'],
    num_workers=0,  # Set to 0 for macOS compatibility
    train_transform=train_transform,
    val_transform=val_transform
)

print("✅ DataLoaders created successfully!")
print(f"\nBatch configuration:")
print(f"  Batch size: {config['training']['batch_size']}")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")
print(f"\nClass names ({len(class_names)}):")
for i, name in enumerate(class_names, 1):
    print(f"  {i:2d}. {name}")

## 5. Model Architecture Design

### 5.1 Transfer Learning Strategy

**Why Transfer Learning?**
- Pre-trained models have learned general visual features from ImageNet (1M+ images)
- Faster convergence with less data
- Better generalization
- Reduced training time

**Approach:**
1. Load pre-trained ResNet50 (trained on ImageNet)
2. Freeze backbone layers (feature extractor)
3. Replace final classification layer
4. Train only the new classifier head
5. Optional: Fine-tune later layers if needed

In [ ]:
# Import model architecture
from models.resnet_model import create_resnet50

# Create model
num_classes = len(class_names)
model = create_resnet50(num_classes=num_classes, pretrained=True)

# Move to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"🏗️ Model Architecture: ResNet50")
print(f"Device: {device}")
print("\n" + "="*70)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Frozen parameters: {total_params - trainable_params:,}")
print(f"Trainable %: {trainable_params/total_params*100:.2f}%")
print("="*70)

# Display model summary
print("\n📋 Model Structure:")
print(model)

## 6. Training Configuration

### 6.1 Loss Function & Optimizer

In [ ]:
# Loss function - CrossEntropyLoss for multi-class classification
criterion = nn.CrossEntropyLoss()

# Optimizer - Adam (adaptive learning rate)
optimizer = optim.Adam(
    model.parameters(),
    lr=config['training']['learning_rate'],
    weight_decay=1e-4  # L2 regularization
)

# Learning rate scheduler - ReduceLROnPlateau
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=5,
    verbose=True
)

print("⚙️ Training Configuration:")
print("="*70)
print(f"Loss function: CrossEntropyLoss")
print(f"Optimizer: Adam")
print(f"  Learning rate: {config['training']['learning_rate']}")
print(f"  Weight decay: 1e-4 (L2 regularization)")
print(f"Scheduler: ReduceLROnPlateau")
print(f"  Factor: 0.5 (reduce by half)")
print(f"  Patience: 5 epochs")
print(f"Early stopping patience: {config['training']['early_stopping_patience']} epochs")
print("="*70)

### 6.2 Training & Validation Functions

Define core training and validation loops with metrics tracking.

In [ ]:
from tqdm.notebook import tqdm

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc='Training', leave=False)
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Track metrics
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc


def validate(model, dataloader, criterion, device):
    """Validate the model"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validating', leave=False)
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Track metrics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Update progress bar
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

print("✅ Training and validation functions defined")

## 7. Model Training

### 7.1 Training Loop with Early Stopping

**Note**: Full training takes ~15 hours on CPU. This notebook demonstrates the training process with a small number of epochs. For full training, use the command-line script: `python src/models/run_training.py`

In [ ]:
# Training configuration for notebook (demo with fewer epochs)
NUM_EPOCHS_DEMO = 3  # For demonstration - change to 50 for full training
CHECKPOINT_DIR = Path.cwd().parent / 'results' / 'models'
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'learning_rates': []
}

# Early stopping
best_val_loss = float('inf')
patience_counter = 0
EARLY_STOP_PATIENCE = config['training']['early_stopping_patience']

print(f"🚀 Starting training for {NUM_EPOCHS_DEMO} epochs...")
print(f"(For full training, use: python src/models/run_training.py)")
print("="*70)

In [ ]:
# Main training loop
for epoch in range(1, NUM_EPOCHS_DEMO + 1):
    print(f"\nEpoch {epoch}/{NUM_EPOCHS_DEMO}")
    print("-" * 70)
    
    # Train
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    # Update learning rate
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['learning_rates'].append(current_lr)
    
    # Print epoch results
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
    print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc*100:.2f}%")
    print(f"Learning Rate: {current_lr:.6f}")
    
    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Save best model
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc
        }, CHECKPOINT_DIR / 'best_model_demo.pth')
        print(f"✅ New best model saved! (Val Loss: {val_loss:.4f})")
    else:
        patience_counter += 1
        print(f"No improvement ({patience_counter}/{EARLY_STOP_PATIENCE})")
        
        if patience_counter >= EARLY_STOP_PATIENCE:
            print(f"\n⚠️ Early stopping triggered after {epoch} epochs")
            break

print("\n" + "="*70)
print(f"✅ Training complete!")
print(f"Best validation loss: {best_val_loss:.4f}")

### 7.2 Training Curves Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

epochs_range = range(1, len(history['train_loss']) + 1)

# Loss plot
axes[0].plot(epochs_range, history['train_loss'], 'b-', label='Train Loss', linewidth=2, marker='o')
axes[0].plot(epochs_range, history['val_loss'], 'r-', label='Val Loss', linewidth=2, marker='s')
axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(epochs_range, [acc*100 for acc in history['train_acc']], 
             'b-', label='Train Acc', linewidth=2, marker='o')
axes[1].plot(epochs_range, [acc*100 for acc in history['val_acc']], 
             'r-', label='Val Acc', linewidth=2, marker='s')
axes[1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

# Learning rate plot
axes[2].plot(epochs_range, history['learning_rates'], 'g-', linewidth=2, marker='^')
axes[2].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
axes[2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[2].set_yscale('log')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Training Insights:")
print(f"  Final train accuracy: {history['train_acc'][-1]*100:.2f}%")
print(f"  Final validation accuracy: {history['val_acc'][-1]*100:.2f}%")
print(f"  Best validation accuracy: {max(history['val_acc'])*100:.2f}%")
if history['val_acc'][-1] > history['train_acc'][-1]:
    print("  ✅ Validation > Training suggests good generalization")
else:
    print("  ⚠️ Training > Validation may indicate overfitting")

## 8. Loading Pre-trained Model

For evaluation and further analysis, load the best model from the full 50-epoch training run.

In [ ]:
# Load the best model from full training
best_model_path = Path.cwd().parent / 'results' / 'models' / 'best_model.pth'

if best_model_path.exists():
    print(f"📥 Loading best model from: {best_model_path}")
    
    # Load checkpoint
    checkpoint = torch.load(best_model_path, map_location=device)
    
    # Get model info from checkpoint
    checkpoint_num_classes = checkpoint['model_state_dict']['model.fc.4.bias'].shape[0]
    
    # Create model with correct number of classes
    eval_model = create_resnet50(num_classes=checkpoint_num_classes, pretrained=False)
    eval_model.load_state_dict(checkpoint['model_state_dict'])
    eval_model = eval_model.to(device)
    eval_model.eval()
    
    print(f"✅ Model loaded successfully!")
    print(f"   Epoch: {checkpoint.get('epoch', 'N/A')}")
    print(f"   Validation Accuracy: {checkpoint.get('val_acc', 0)*100:.2f}%")
    print(f"   Validation Loss: {checkpoint.get('val_loss', 0):.4f}")
else:
    print(f"⚠️ Best model not found at {best_model_path}")
    print(f"   Using demo model for evaluation")
    eval_model = model
    eval_model.eval()

## 9. Quick Evaluation Check

Let's do a quick accuracy check on the test set. For detailed analysis, see `03_results_analysis.ipynb`.

In [ ]:
# Quick evaluation on test set
def quick_evaluate(model, dataloader, device):
    """Quick accuracy check"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc='Evaluating'):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = outputs.max(1)
            
            total += labels.size(0)
            correct += (preds.cpu() == labels).sum().item()
    
    return correct / total

# Run quick evaluation
print("🔍 Quick Test Set Evaluation...")
test_accuracy = quick_evaluate(eval_model, test_loader, device)

print(f"\n✅ Test Accuracy: {test_accuracy*100:.2f}%")
print(f"\n📊 For detailed analysis:")
print(f"   - Confusion matrices")
print(f"   - Per-class metrics")
print(f"   - Error analysis")
print(f"   - Confidence distributions")
print(f"\n→ See notebook: 03_results_analysis.ipynb")

## 10. Training Summary & Next Steps

### What We Built

**Model Configuration:**
- ✅ ResNet50 with transfer learning
- ✅ Custom classification head (16 classes)
- ✅ Only ~4% parameters trainable (efficient)

**Training Strategy:**
- ✅ Adam optimizer with learning rate scheduling
- ✅ Advanced data augmentation
- ✅ Early stopping to prevent overfitting
- ✅ Validation > Training accuracy (good generalization)

**Key Insights:**
1. Transfer learning dramatically speeds up convergence
2. Data augmentation prevents overfitting effectively
3. ReduceLROnPlateau helps escape local minima
4. Model generalizes well to validation data

---

### Next Steps

**For Detailed Analysis** → `03_results_analysis.ipynb`:
- Comprehensive evaluation metrics
- Confusion matrices and error analysis
- Per-class performance breakdown
- Prediction confidence analysis
- Model interpretation and insights
- Production readiness assessment